In [1]:
import sys
sys.path.append('./../src')

In [141]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
from bs4 import BeautifulSoup
import requests
import web_scraping as ws
import re
from importlib import reload
from pymongo import MongoClient

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [21]:
# This piece of code saves me so much time! 
reload(ws)

<module 'web_scraping' from './src/web_scraping.py'>

## Scraping Gaysian Diaries ##
Parsing steps:
1. Get all the blog post urls by navigating through each GD page (there are currently 4 pages). Put them in a list.
2. Parse through each blog post and collect the title, post date, and the paragraphs. The
3. Save the titles, post dates in a list of dicts and the paragraphs in .txt files. 

In [119]:
gd_home_page = 'https://gaysiandiaries.com/'

In [120]:
gd_blog_dicts = ws.gd_get_blog_dicts(gd_home_page)

In [186]:
gd_blog_dicts[0]

{'title': 'Diary Entry #1',
 'date': 'March 1st, 2016',
 'url': 'https://gaysiandiaries.com/post/140302789153/diary-entry-1',
 'text': 'Dear Diary,\n\nI spend a lot of time analyzing how much our past experiences affect who we are today. Growing up, my father wasn’t around much. He did business internationally while my mom took care of us at home. My parents were always supportive of me and my sisters. My dad was endearing but was never a man of many words. My mother cared for us so much and always wanted the best for us. Every night before bed, my mom would hold my hands and tell me she loved me. She made sure I knew that every night before I fell asleep.\n\nWhen I was 8, my mom found out my father was having an affair during his Asia business trips. I don’t know how she found out, but she somehow obtained a photo of the mistress and some letters that my dad had written to her.\xa0I noticed my mom staying up late at night crying, obsessively staring at the evidence that my father was 

In [127]:
with open('../pickles/gd_blog_dicts.pickle', 'wb') as to_write:
    pickle.dump(gd_blog_dicts, to_write)

## Scraping G3S ##

In [5]:
g3s_post_directory_page = 'https://gaysianthirdspace.tumblr.com/tags'
g3s_soup = ws.create_soup(g3s_post_directory_page)

In [64]:
g3s_all_blog_urls = ws.g3s_get_blog_urls(g3s_tag_page_urls)

In [117]:
reload(ws)

<module 'web_scraping' from './../src/web_scraping.py'>

In [67]:
with open('../pickles/g3s_blog_urls.pickle', 'wb') as to_write:
    pickle.dump(g3s_all_blog_urls, to_write)

In [128]:
g3s_blog_dicts = ws.g3s_get_blog_dicts(g3s_all_blog_urls)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [183]:
g3s_blog_dicts[-1]

{'title': 'Follower Friday: gregasaurus',
 'date': 'Jan 8th, 2016',
 'url': 'https://gaysianthirdspace.tumblr.com/post/136911555225/follower-friday-gregasaurus',
 'text': 'Follower Fridays is a series of profiles highlighting members of Gaysian Third Space to showcase the diversity of gaysians in the Community. This week’s featured member is @gregasaurus\n\nWho are you?\n\n\n\nWow, this feels like filling out an OK Cupid profile. Anywho, if you couldn’t pick it up from my carefully encrypted Tumblr handle, my name is Greg. I’m 25, which in both Tumblr and gay years makes me a fossil (though I still don’t qualify for that discount at IHOP). I like writing, taking pictures of and/or singing love ballads to my food, and long walks to basically anywhere since I don’t drive. I appreciate irreverent humor, puns, and just general weirdness that most people are afraid to embrace within themselves. I feel like honoring your weirdness and quirks is akin to coming out–it allows you to be yourself

In [140]:
with open('../pickles/g3s_blog_dicts.pickle', 'wb') as to_write:
    pickle.dump(g3s_blog_dicts, to_write)

## Uploading Scraped Data to MongoDB ##

In [142]:
client = MongoClient()
db = client.blogs
db.list_collection_names()

[]

In [146]:
db.gd.insert_many(gd_blog_dicts)

In [164]:
cursor = db.gd.find({}, {'_id': 0, 'title': 1, 'date': 1, 'url': 1, 'text': 1})

In [144]:
db.g3s.insert_many(g3s_blog_dicts)

In [166]:
cursor = db.g3s.find({}, {'_id': 0, 'title': 1, 'date': 1, 'url': 1, 'text': 1})

In [177]:
cursor = db.g3s.find({'date': 'Apr 4th, 2016'}, {'_id': 0, 'title': 1, 'date': 1, 'url': 1, 'text': 1})

In [178]:
list(cursor)

[{'title': None,
  'date': 'Apr 4th, 2016',
  'url': 'https://gaysianthirdspace.tumblr.com/post/142262150964/heyarnie-i-dont-usually-write-sappy-friendship',
  'text': 'heyarnie:\n\nI don’t usually write sappy friendship posts, but I’m feeling sentimental tonight so I’ll break the rules.\n\nLet me tell you the story about *Gumby\n\nI started following this guy, three years ago when I was depressed and lonely.\n\nI had no gay friends back then and barely had any gay experiences.\n\nHe was known as Gumbyiscool.\xa0His writing was beautiful and his stories were interesting. Some of them were so familiar to me, I could have sworn that I had written them myself.\n\nOver time, I grew attached to that blog. I looked forward to reading those daily posts, and reading them actually nade me feel less lonely. It made me feel like someone out there was going through the same thing.\n\nThen he deleted his blog and disappeared off the face of Tumblr. I was devestated. It baffled me - why was I so dep

In [184]:
pipeline = [{'$match': {'title': 'Diary Entry #33: How real are dreams?'}}, {'$count': 'test_count'}]

list(db.gd.aggregate(pipeline))

[{'test_count': 1}]